In [ ]:
!pip install transformers torch accelerate bitsandbytes huggingface_hub fastapi uvicorn python-multipart
!pip install ngrok

In [ ]:
from huggingface_hub import login
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSeq2SeqLM
import os
from ngrok import ngrok
from fastapi import FastAPI
import uvicorn
import nest_asyncio
import asyncio
from pydantic import BaseModel

In [ ]:
login('HUGGING FACE LOGIN')
ngrok.set_auth_token('NGROK AUT TOKEN')

In [ ]:
class TranslateModel:
  def __init__(self):
    self.model_id = 'google/madlad400-3b-mt'
    self.model = None
    self.tokenizer = None
    self.bits_config = BitsAndBytesConfig(load_in_4bit=True,
                                          bnb_4bit_quant_type="nf4",
                                          bnb_4bit_use_double_quant=True,
                                          bnb_4bit_compute_dtype=torch.bfloat16,
                                          llm_int8_threshold=6.0,
                                          llm_int8_enable_fp32_cpu_offload=True)

  def init_llm(self):
    self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_id, quantization_config=self.bits_config, device_map='auto', trust_remote_code=True)
    self.tokenizer = AutoTokenizer.from_pretrained(self.model_id, trust_remote_code=True)

  def translate(self, text, lang):
    prompt = f'<2{lang}> {text}'
    input_ids = self.tokenizer(prompt, return_tensors='pt').input_ids.to(self.model.device)
    outputs = self.model.generate(input_ids, max_new_tokens=512)
    return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
app = FastAPI()
translate_model = TranslateModel()
translate_model.init_llm()

In [ ]:
class TranslationRequest(BaseModel):
  items: list

In [ ]:
@app.get('/')
async def init_api():
  return {'message': 'Api Ativa'}

@app.post('/translate')
async def translate_endpoint(request: TranslationRequest):
  translated_list = []
  for items in request.items:
    translated_text = translate_model.translate(items['text'], items['lang'])
    translated_list.append(translated_text)
  return {'translations': translated_list}

In [ ]:
nest_asyncio.apply()
async def setup_ngrok():
  tunnel = await ngrok.connect(9002)
  return tunnel

async def run_server():
  config = uvicorn.Config(app, host='0.0.0.0', port=9002)
  server = uvicorn.Server(config)
  await server.serve()

asyncio.run(asyncio.gather(setup_ngrok(), run_server()))